In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import random
import datetime

In [2]:
# https://archive.ics.uci.edu/ml/datasets/auto+mpg
autompg = pd.read_csv('auto_mpg.csv')
autompg_disp = autompg['displacement'].astype(float)
autompg_mpg = autompg['mpg'].astype(float)
mean_disp = np.mean(autompg_disp)
min_disp = np.min(autompg_disp)
max_disp = np.max(autompg_disp)
print(mean_disp, max_disp, min_disp, max_disp-min_disp)
autompg_disp = autompg_disp.apply(lambda x:x/100)
print(autompg_disp.describe())
autompg_mpg = autompg_mpg.apply(lambda x: x/100)
print(autompg_mpg.describe())

193.42587939698493 455.0 68.0 387.0
count    398.000000
mean       1.934259
std        1.042698
min        0.680000
25%        1.042500
50%        1.485000
75%        2.620000
max        4.550000
Name: displacement, dtype: float64
count    398.000000
mean       0.235146
std        0.078160
min        0.090000
25%        0.175000
50%        0.230000
75%        0.290000
max        0.466000
Name: mpg, dtype: float64


In [3]:
learning_rate = 0.003
n_epochs = 3000
display_step = 100
train_X = np.asarray(autompg_disp)
train_Y = np.asarray(autompg_mpg)
n_samples = train_X.shape[0]

In [4]:
def getbatch(xval, yval, arraylength, batchsize=30):
    count = 0 
    while count < arraylength/batchsize:
        random.seed(datetime.datetime.now())
        randstart = random.randint(0, arraylength-batchsize-1)
        count += 1
        yield (xval[randstart:randstart+batchsize], yval[randstart:randstart+batchsize])

# Test
#for i in getbatch(train_X, train_Y, n_samples):
#    print(i)

In [5]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W = tf.Variable(np.random.randn())
b = tf.Variable(np.random.randn())

# predicted is X*W+b. 
pred = tf.add(tf.multiply(X, W), b)

# The cost function is ((predicted-actual)^2)/2*n_samples. 
cost = tf.reduce_sum(tf.pow(pred-Y, 2))/(2*n_samples)

optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Add all ops that need to be initialized
# The initilization needs to be run only after session is created
# as in session.run(init) below.
init = tf.global_variables_initializer()

# Summary writer and its configuration
writer = tf.summary.FileWriter("./logs")
tf.summary.scalar("Cost", cost)
merged_summary_op = tf.summary.merge_all()

In [6]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for (x, y) in getbatch(train_X, train_Y, n_samples):
            _, costval, merged_summary = sess.run([optimizer, cost, merged_summary_op], 
                                                  feed_dict={X: x, Y: y})
        writer.add_summary(merged_summary, epoch)
        if (epoch+1) % display_step == 0:
            c = sess.run(cost, feed_dict={X: train_X, Y:train_Y})
            print("Epoch = {:04d} and cost = {:.9f}".format(epoch+1, c))

    training_cost = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
    finalW = sess.run(W)
    finalb = sess.run(b)
    print("The final W = %0.4f and b = %0.4f" %(finalW, finalb))

Epoch = 0100 and cost = 0.526417911
Epoch = 0200 and cost = 0.438049614
Epoch = 0300 and cost = 0.388165623
Epoch = 0400 and cost = 0.345071405
Epoch = 0500 and cost = 0.306156367
Epoch = 0600 and cost = 0.271642059
Epoch = 0700 and cost = 0.241282418
Epoch = 0800 and cost = 0.214207590
Epoch = 0900 and cost = 0.190035298
Epoch = 1000 and cost = 0.168596506
Epoch = 1100 and cost = 0.149639815
Epoch = 1200 and cost = 0.132979482
Epoch = 1300 and cost = 0.118245579
Epoch = 1400 and cost = 0.105099902
Epoch = 1500 and cost = 0.093555912
Epoch = 1600 and cost = 0.083107658
Epoch = 1700 and cost = 0.073789187
Epoch = 1800 and cost = 0.065613478
Epoch = 1900 and cost = 0.058382567
Epoch = 2000 and cost = 0.051937491
Epoch = 2100 and cost = 0.046206564
Epoch = 2200 and cost = 0.041139115
Epoch = 2300 and cost = 0.036662053
Epoch = 2400 and cost = 0.032622930
Epoch = 2500 and cost = 0.029079694
Epoch = 2600 and cost = 0.025937488
Epoch = 2700 and cost = 0.023155289
Epoch = 2800 and cost = 0.02

## Opening tensorboard

In the command line, type "tensorboard --logdir=./logs"
This will launch a local webserver at port 6006. Visit http://localhost:6006 in a browser. Then click on the label 'Cost' to see the graph.  You can make the graph larger by clicking on the rectangular button at the bottom left corner of the graph. You can zoom in to the graph by drawing a box around the region to zoom in. You can zoom out by double clicking the graph.